# Projeto 1 - Ciência dos Dados

Nome: _____

Nome: _____

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [8]:
%matplotlib inline
!pip install emoji
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from emoji import UNICODE_EMOJI

  Created wheel for emoji: filename=emoji-0.6.0-py3-none-any.whl size=49720 sha256=3e808bb6e5cf0feca9bdd8c22ec965a191237371006942ddb8eeed8e28830efe
  Stored in directory: c:\users\lava boy 2.0\appdata\local\pip\cache\wheels\4e\bf\6b\2e22b3708d14bf6384f862db539b044d6931bd6b14ad3c9adc
Successfully built emoji


**Em `filename`, coloque o nome do seu arquivo de dados!**

In [9]:
import os

filename = 'rtx 3080.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo rtx 3080.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [10]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Classificação
0,@rtx_3080 @thelockalarm don’t let em get frost...,I
1,answering your rtx 3080 questions! - probing p...,R
2,are you still doing my mom https://t.co/yn0umv...,I
3,i guess us customs wins this round. a partner ...,P
4,gonna order when listing on amazon us!\n \nhtt...,I


In [11]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste
0,@kaylasweetx i just want a rtx 3080
1,plan on upgrading with @nvidia's rtx 3080? be ...
2,@insomniacgames i thought the playstation 5 co...
3,@asus_rog @asus_rog are you guys releasing the...
4,@vixlyt @eryxies @charlieintel its 2020. let's...


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [12]:
def separate_emoji(text): #separa emojis dos outros caracteres
    return ''.join(' ' + char if char in UNICODE_EMOJI else char for char in text).strip() #caso seja um emoji, adiciona um espaco, caso nao seja, apenas retorna a letra.

In [35]:
chars = [",", "'", '"', ".", "!", "?", ";", ":", "[", "]","{", "}", "(", ")", "#", "/",  "\\", "https","+", "@", "*", "-"] 
word_count = dict()
for i, tweet in train.iterrows(): #iterando a base de dados
    piu = tweet.Treinamento #carregando apenas os tweets
    
    for c in chars: #iterando a listad e cacarteres p/ remover
        piu = piu.replace(c, " ") #trocando os caracteres por " "
        
    for word in separate_emoji(piu).split(): #splitando os tweets em palavras e emojis separadamente
        word_count[word] = word_count.get(word, {'I':0, 'P':0, 'R':0}) #garantindo que  a palavra existe no dicionario, caso nao exista ela eh criada
        word_count[word][tweet.Classificação.upper()]+=1 #incrementando o contador do valor correto
        
I = {word: word_count[word]['I'] for word in word_count if word_count[word]['I'] > 0} #I contem todas as palavras Irrelevantes, seu sum(I.values) == total de palavras Irrelevantes
P = {word: word_count[word]['P'] for word in word_count if word_count[word]['P'] > 0} #P contem todas as palavras Pouco relevantes, seu sum(P.values) == total de palavras Pouco relevantes
R = {word: word_count[word]['R'] for word in word_count if word_count[word]['R'] > 0} #R contem todas as palavras Relevantes, seu sum(R.values) == total de palavras Relevantes

4437
9819


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [56]:

total = len(word_count) #total de palavras no banco de dados
totalR = (sum(R.values())) #total de palavaras na categoria Relevante
totalP = (sum(P.values())) #total de palavaras na categoria Pouco Relevante
totalI = (sum(I.values())) #total de palavaras na categoria Irrelevante

l = [] #lista de classificacoes, mesma ordem da base test
for i, tweet in test.iterrows(): #iterando a base de dados
    
    #valores para a probabiliade do tweet ser Irrelevante, comeca como 1 para nao der erro na multiplicacao abaixo
    i = 1
    p = 1
    r = 1
    
    piu = tweet.Teste #carregando apenas os tweets 
    
    for c in chars: #iterando a listad e cacarteres p/ remover
        piu = piu.replace(c, " ") #trocando os caracteres por " "
    
    #splitando os tweets em palavras e emojis separadamente e calculando a probalidade de cada palavra de ser cada categoria, fazendo a suavizacao de laplace
    for word in separate_emoji(piu).split(): 
        
        #calculando a probabiliade de ser relevamte, pegamos a probabiliadde total e multiplicamos pelo valor da palavra
        r *= (R.get(word, 0) + 1) / (totalR + total) 
        p *= (P.get(word, 0) + 1) / (totalP + total) 
        i *= (I.get(word, 0) + 1) / (totalI + total) 
        
    result = {"R":r,"P":p,"I": i} #dicionario para a comparacao dentro do MAX abaixo
    l.append(max(result, key = result.get)) #compara os valores de r, p, i, e adiciona correta a lista L 

test["Classificação"] = l #adiciona a coluna classificacao a test, como a ordem eh a mesma, podemos adicionar.

test.head(20)

['P', 'R', 'R', 'P', 'R', 'P', 'P', 'R', 'R', 'P', 'I', 'P', 'R', 'R', 'I', 'R', 'R', 'P', 'P', 'I', 'P', 'R', 'P', 'R', 'R', 'I', 'I', 'R', 'R', 'P', 'P', 'P', 'R', 'R', 'P', 'R', 'P', 'I', 'R', 'R', 'R', 'P', 'I', 'I', 'R', 'P', 'I', 'P', 'P', 'P', 'R', 'P', 'P', 'P', 'P', 'R', 'I', 'P', 'P', 'P', 'R', 'P', 'P', 'P', 'R', 'R', 'P', 'I', 'R', 'P', 'R', 'P', 'I', 'I', 'P', 'R', 'R', 'R', 'I', 'P', 'I', 'R', 'P', 'R', 'P', 'R', 'I', 'R', 'R', 'R', 'P', 'I', 'R', 'P', 'I', 'R', 'R', 'R', 'P', 'R', 'P', 'R', 'R', 'P', 'P', 'R', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'R', 'R', 'P', 'P', 'R', 'R', 'R', 'P', 'P', 'R', 'R', 'R', 'R', 'R', 'I', 'R', 'R', 'I', 'P', 'P', 'P', 'P', 'P', 'R', 'P', 'P', 'R', 'I', 'R', 'R', 'P', 'P', 'P', 'I', 'P', 'R', 'R', 'R', 'R', 'P', 'R', 'R', 'R', 'P', 'I', 'R', 'P', 'P', 'I', 'P', 'R', 'P', 'R', 'P', 'R', 'I', 'R', 'R', 'R', 'P', 'P', 'R', 'I', 'P', 'I', 'R', 'P', 'P', 'P', 'R', 'I', 'P', 'I', 'R', 'P', 'R', 'P', 'R', 'R', 'R', 'P', 'I', 'R', 'I', 'P', 'R', 'R',

,Teste,Classificação
0,@kaylasweetx i just want a rtx 3080,P
1,plan on upgrading with @nvidia's rtx 3080? be ...,R
2,@insomniacgames i thought the playstation 5 co...,R
3,@asus_rog @asus_rog are you guys releasing the...,P
4,@vixlyt @eryxies @charlieintel its 2020. let's...,R
5,@datbx_ i am not cool 😭,P
6,@dnpthree @newcuiture #freegamingpc cad to usd...,P
7,@colteastwood i’m getting the rtx 3080,R
8,@greencapt @chuckxl @bigmaki this all just see...,R
9,@flaw4q this going to be on my rtx 3080 i9 109...,P


In [50]:
x = ""
n = 1
while os.path.isfile('./{0}.xlsx'.format("3080" + x)): #salva um novo arquivo de excel para cada iteracao do loop, com n+1 por iteracao, para comapar as iteracoes do classificador 
    x = "-" + str(n)
    n += 1
file = './{0}.xlsx'.format("3080" + x)
    

writer = pd.ExcelWriter(file) #Abre o arquivo para escrita

test.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False) #salva o arquivo teste classificado


writer.save() #fecha o arquivo

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**